In [6]:
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import re
import os
import time



In [2]:
data=pd.read_pickle('scrape_data.pkl')
data1=data[0:200] # FÆRDIG
data2=data[200:1000]      # CHRISTIAN
data3=data[1000:2000]     # CHRISTIAN
data4=data[2000:3000]
data5=data[3000:4000]     # THOR
data6=data[4000:]         # MORTEN

In [3]:
def skyscannerscraper(dep,arr,date, save, driverpath, direct='true'):
    # Function scraping skyscanner.dk
    # dep: airport code for departure airport
    # arr: airport code for arrival airport
    # date: date of flight, format YYMMDD
    # save: filename
    # driverpath: complete path to driver
    # direct: indicates whether direct flight is preferred
    save2=save+".pkl"
    
    driver = webdriver.Firefox(executable_path=driverpath)
    driver.implicitly_wait(7)
    wait = WebDriverWait(driver, 8)
    urlbase = 'https://www.skyscanner.com/transport/flights/'
    urlpas='/?adults=1&children=0&adultsv2=1&childrenv2=&infants=0&cabinclass=economy&rtn=0&preferdirects='
    urlend = '&outboundaltsenabled=false&inboundaltsenabled=false&ref=home#results'
    data = []
    count=0
    for depa,arri in zip(dep,arr):
        count+=1
        url = urlbase + str(depa) + str('/') + str(arri) + str('/')+ str(date) + urlpas + direct + urlend

        driver.get(url)
        time.sleep(6) 
        try:
            wait.until(EC.presence_of_element_located((By.CLASS_NAME, "filter-sub")))
            wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "label-text")))
            #wait.until(EC.visibility_of_element_located((By.CLASS_NAME,"day-list-count")))
            price = [price.text for price in driver.find_elements(By.CLASS_NAME, "filter-sub")]
            label = [label.text for label in driver.find_elements(By.CLASS_NAME, "label-text")]
            r=price[0]
            if r==' ':
                del r
                #print('r=0')
                raise ValueError
            elif r=='Ingen':
                #print('r=0')
                raise ValueError
                del r
            else:
                data.append({
                    "price": price[0],
                    "origin": str(depa),
                    "destination": str(arri),
                    "label": label[0],
                    "day":0
                        })
                del r
                del price
            
        except (TimeoutException,StaleElementReferenceException,BaseException):
            try:
                WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.bpk-close-button-65MQ0'))).click()
                wait.until(EC.presence_of_element_located(By.CLASS_NAME, "filter-sub"))
                wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "label-text")))
                #wait.until(EC.visibility_of_element_located((By.CLASS_NAME,"day-list-count")))
                price = [price.text for price in driver.find_elements(By.CLASS_NAME, "filter-sub")]
                label = [label.text for label in driver.find_elements(By.CLASS_NAME, "label-text")]
                r=price[0]
                if r==' ':
                    del r
                    #print('r=0')
                    raise ValueError
                elif r=='Ingen':
                    del r
                    #print('r=0')
                    raise ValueError
                else:
                    data.append({
                    "price": price[0],
                    "origin": str(depa),
                    "destination": str(arri),
                    "label": label[0],
                    "day":0
                        })
                    del r
                    del price
                    


            except (TimeoutException,StaleElementReferenceException,BaseException):
                try:
                    time.sleep(3)
                    #print('Round2')
                    wait.until(EC.presence_of_element_located((By.CLASS_NAME, "filter-sub")))
                    wait.until(EC.presence_of_element_located((By.CLASS_NAME, "label-text")))
                    #wait.until(EC.visibility_of_element_located((By.CLASS_NAME,"day-list-count")))
                    price = [price.text for price in driver.find_elements(By.CLASS_NAME, "filter-sub")]
                    label = [label.text for label in driver.find_elements(By.CLASS_NAME, "label-text")]
                    r=price[0]
                    if r==' ':
                        del r
                        #print('r=0')
                        raise ValueError
                    elif r=='Ingen':
                        #print('r=0')
                        raise ValueError
                        del r
                    else:
                        data.append({
                            "price": price[0],
                            "origin": str(depa),
                            "destination": str(arri),
                            "label": label[0],
                            "day":0
                                })
                        del r
                        del price

                except (TimeoutException,StaleElementReferenceException,BaseException):
                        try:
                            WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.bpk-close-button-65MQ0'))).click()
                            wait.until(EC.presence_of_element_located(By.CLASS_NAME, "filter-sub"))
                            wait.until(EC.presence_of_element_located((By.CLASS_NAME, "label-text")))
                            #wait.until(EC.visibility_of_element_located((By.CLASS_NAME,"day-list-count")))
                            price = [price.text for price in driver.find_elements(By.CLASS_NAME, "filter-sub")]
                            label = [label.text for label in driver.find_elements(By.CLASS_NAME, "label-text")]
                            r=price[0]
                            if r==' ':
                                del r
                                #print('r=0')
                                raise ValueError
                            elif r=='Ingen':
                                del r
                                #print('r=0')
                                raise ValueError
                            else:
                                data.append({
                                "price": price[0],
                                "origin": str(depa),
                                "destination": str(arri),
                                "label": label[0],
                                "day":0
                                    })
                                del r
                                del price

                        except (TimeoutException,StaleElementReferenceException,BaseException):
                            try:
                                #print('indirect')
                                wait.until(EC.presence_of_element_located((By.CLASS_NAME, "filter-sub")))
                                wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "label-text")))

                                price = [price.text for price in driver.find_elements(By.CLASS_NAME, "filter-sub")]
                                label = [label.text for label in driver.find_elements(By.CLASS_NAME, "label-text")]
                                data.append({
                                        "price": price[1],
                                        "origin": str(depa),
                                        "destination": str(arri),
                                        "label": label[1],
                                        "day":0,
                                        "alternative": price[0]
                                            })
                                del price


                            except (TimeoutException,StaleElementReferenceException,BaseException):

                                print("Not working for "+str(url))


                                data.append({
                                        "price": '-',
                                        "origin": str(depa),
                                        "destination": str(arri),
                                        "label": '-',
                                        "day":0
                                            })
                    
        print(str(depa)+"-"+str(arri))
        
        print(str(count)+" af " +str(len(dep.index)))
        time.sleep(2)
        price_data = pd.DataFrame(data)
        if count%20==0:
            price_data.to_pickle(save2)
        if count%200==0:
            time.sleep(300)
    driver.close()
    
    
    
    return price_data

In [587]:
# Indsæt driverpath (skal vise lokal path)

prices2=skyscannerscraper(data2['Origin'],data2['Dest'],190527,save='prices_200_1000', driverpath='/Users/Christian/Dropbox/Universitet/TSODAS/TSDS-gruppe-2019/Exam/geckodriver', direct='true')



BWI-LAX
1 af 2
Round2
BWI-LIT
2 af 2


### MORTEN

In [ ]:
# Indsæt driverpath (skal vise lokal path)
prices2=skyscannerscraper(data6['Origin'],data6['Dest'],190527,save='prices_4000_', driverpath='C:/Users/Morte/Documents/GitHub/TSDS-gruppe-2019/Exam/geckodriver_win64', direct='true')
prices2.to_pickle("prices_4000_.pkl")

DFW-SAN
1 af 1342
PSP-ORD
2 af 1342
MSY-MIA
3 af 1342
MFE-DFW
4 af 1342
STL-DFW
5 af 1342
DFW-IND
6 af 1342
DFW-PDX
7 af 1342
DFW-SEA
8 af 1342
RIC-DFW
9 af 1342
DFW-JAX
10 af 1342
DFW-ELP
11 af 1342
MIA-SJU
12 af 1342
ORD-PSP
13 af 1342
STL-SAN
14 af 1342
SJC-AUS
15 af 1342
MSY-STL
16 af 1342
PSP-DFW
17 af 1342
RNO-DFW
18 af 1342
IAD-DFW
19 af 1342
DFW-SJC
20 af 1342
BNA-LGA
21 af 1342
FAT-DFW
22 af 1342
Not working for https://www.skyscanner.com/transport/flights/BDL/DFW/190527/?adults=1&children=0&adultsv2=1&childrenv2=&infants=0&cabinclass=economy&rtn=0&preferdirects=true&outboundaltsenabled=false&inboundaltsenabled=false&ref=home#results
BDL-DFW
23 af 1342
IND-DFW
24 af 1342
MIA-IAH
25 af 1342
ORF-DFW
26 af 1342
TUS-ORD
27 af 1342
RSW-DFW
28 af 1342
DFW-EWR
29 af 1342
BWI-DFW
30 af 1342
BHM-DFW
31 af 1342
STL-LGA
32 af 1342
AUS-SJC
33 af 1342
DFW-FLL
34 af 1342
DFW-BDL
35 af 1342
SNA-DFW
36 af 1342
ORD-RNO
37 af 1342
FLL-SJU
38 af 1342
DFW-SJU
39 af 1342
JAC-DFW
40 af 1342
ORD-MSY

TUS-JFK
329 af 1342
EWR-JFK
330 af 1342
LGB-ORD
331 af 1342
ORD-LGB
332 af 1342
JFK-SWF
333 af 1342
PBI-SWF
334 af 1342
SWF-PBI
335 af 1342
IAH-LAX
336 af 1342
IAH-SAN
337 af 1342
IAH-SMF
338 af 1342
IAH-SNA
339 af 1342
PDX-IAH
340 af 1342
SAT-IAH
341 af 1342
LAX-IAH
342 af 1342
SEA-IAH
343 af 1342
CLE-IAH
344 af 1342
PDX-EWR
345 af 1342
IAH-EWR
346 af 1342
RSW-CLE
347 af 1342
IAH-PHL
348 af 1342
SMF-IAH
349 af 1342
SNA-EWR
350 af 1342
HNL-IAH
351 af 1342
EWR-IAH
352 af 1342
EWR-AUS
353 af 1342
EWR-SAN
354 af 1342
IAH-AUS
355 af 1342
IAH-JFK
356 af 1342
BWI-IAH
357 af 1342
LGA-IAH
358 af 1342
IAH-SJU
359 af 1342
IAH-FLL
360 af 1342
IAH-BWI
361 af 1342
SAN-IAH
362 af 1342
AUS-IAH
363 af 1342
MTJ-EWR
364 af 1342
EWR-CLE
365 af 1342
OAK-IAH
366 af 1342
IAH-LGA
367 af 1342
CLE-EWR
368 af 1342
PHL-IAH
369 af 1342
Not working for https://www.skyscanner.com/transport/flights/EWR/STT/190527/?adults=1&children=0&adultsv2=1&childrenv2=&infants=0&cabinclass=economy&rtn=0&preferdirects=true&outbou

### THOR

In [ ]:
# Indsæt driverpath (skal vise lokal path)
prices2=skyscannerscraper(data5['Origin'],data5['Dest'],190527,save='prices_3000_3999', driverpath='C:/Users/thorn/OneDrive/Dokumenter/GitHub/TSDS-gruppe-2019/Exam/geckodriver_win64', direct='true')


